In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Segment Anything Model(SAM)

# Tumors Segmentation with SAM

Install Segment-Anything

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
!pip install opencv-python pycocotools matplotlib onnxruntime onnx

# Import library

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import sys
sys.path.append("..")

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "/kaggle/input/segment-anything-models/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.9,
    stability_score_thresh=0.9,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=120,  # Requires open-cv to run post-processing
)

predictor = SamPredictor(sam)


# Functions

In [ ]:
def show_anns(anns, axes=None):
    if len(anns) == 0:
        return
    if axes:
        ax = axes
    else:
        ax = plt.gca()
        ax.set_autoscale_on(False)
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.5)))
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='red', marker='o', s=80, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='o', s=80, edgecolor='white', linewidth=1.25)
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=1)
    else:
        color = np.array([200/255, 0/255, 0/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
def custom_plot(title,image,specific_point):
    input_label = np.array([1])
    #--------------
    masks = mask_generator_2.generate(image)
    #--------------
    predictor.set_image(image)
    masks_p, scores, logits = predictor.predict(
    point_coords=specific_point,
    point_labels=input_label,
    multimask_output=True,)
    #--------------
    fig, (ax1, ax2, ax3,ax4) = plt.subplots(1, 4, figsize=(13,5))
    #fig.suptitle(f'{title} tumor')
    plt.axis('off')
    ax1.imshow(image)
    ax1.title.set_text("Image")
    ax2.imshow(image)
    ax2.title.set_text("Image+Masks")
    show_anns(masks, ax2)
    for i, (mask, score) in enumerate(zip(masks_p, scores)):
        if i==0:
            ax3.imshow(image)
            ax3.title.set_text("a specific object")
            show_points(specific_point, input_label, ax3)
            show_mask(mask, ax4)
            ax4.title.set_text(f"Mask - Score: {score:.3f}")
    for ax in fig.get_axes():
        ax.label_outer()
        ax.axis('off')

# 1.Liver

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/sam-images/Tumor/Liver/Liver01.jpg'), cv2.COLOR_BGR2RGB)
specific_point = np.array([[50, 100]])
custom_plot("Liver",image,specific_point)

# 2.Brain

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/sam-images/Tumor/Brain/Brain tumor01.jpg'), cv2.COLOR_BGR2RGB)
specific_point = np.array([[175, 175]])
custom_plot("Brain",image,specific_point)

# 3.Lung

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/sam-images/Tumor/Lung/Lung01.jpg'), cv2.COLOR_BGR2RGB)
specific_point = np.array([[200, 250]])
custom_plot("Lung",image,specific_point)

# 4.Kidney

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/sam-images/Tumor/Kidney/kidney04.jpg'), cv2.COLOR_BGR2RGB)
specific_point = np.array([[215, 150]])
custom_plot("Kidney",image,specific_point)

# X-Ray image Segmentation 

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/pneumonia-xray-images/train/opacity/person1000_virus_1681.jpeg'), cv2.COLOR_BGR2RGB)
specific_point = np.array([[900, 950]])
custom_plot("X-Ray-Lung",image,specific_point)

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/pneumonia-xray-images/train/opacity/person1000_virus_1681.jpeg'), cv2.COLOR_BGR2RGB)
specific_point = np.array([[250, 250]])
custom_plot("X-Ray-Lung",image,specific_point)

In [ ]:
image.shape

# Kidney stone images

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/kidney-stone-images/train/images/1-3-46-670589-33-1-63700700749865510700001-5062181202000819812_png_jpg.rf.74493ef7bdab6de49e88f708a5745000.jpg'), cv2.COLOR_BGR2RGB)
specific_point = np.array([[100, 150]])
custom_plot("X-Ray-Lung",image,specific_point)